# Regex Applications

### José Pablo Kiesling Lange

In [1]:
import pandas as pd
import re

In [2]:
dataset = pd.read_csv('pii_dataset.csv')

In [3]:
dataset.head()

,Nombre,Correo,Teléfono,Dirección,Identificación,Texto
0,Reyes Pinilla Rosado,woliver@tena.es,+34967 400 699,"Calle Rambla de Nieves Dalmau, 126, Ávila",40955969R,"Mi nombre es Reyes Pinilla Rosado, puedes cont..."
1,Evita Solsona Escobar,nicolaspou@hernandez.com,991740961,"Calle Vial Luís Martín, 131, Las Palmas",34970246V,"Mi nombre es Evita Solsona Escobar, puedes con..."
2,Lorenza Mate Bayón,jimena05@iglesias.com,+34 673931008,"Alameda Amador Calleja 13 Piso 1 , León, 91297",P5293902,"Mi nombre es Lorenza Mate Bayón, puedes contac..."
3,Renato González-Araujo,tomasalondra@higueras.es,916543174,"Piso 5, Ávila",Y6585174Z,"Mi nombre es Renato González-Araujo, puedes co..."
4,Chus Alejandro Duarte Mayoral,iferrando@peiro.es,+34 650500564,"Urbanización Eliseo Naranjo 68 Piso 8 , Ciudad...",P4418638,"Mi nombre es Chus Alejandro Duarte Mayoral, pu..."


In [4]:
def tokenize(text):
    return text.split()

In [5]:
def token_count(text):
    return len(tokenize(text))

In [6]:
def get_max_token_count_by_column(column_name):
    return dataset[column_name].astype(str).apply(token_count).max()

In [7]:
def get_min_token_count_by_column(column_name):
    return dataset[column_name].astype(str).apply(token_count).min()

## Definición de Regex

In [8]:
ALPHABET_RE = re.compile(r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ]")

### Nombre

En este caso, se puede apreciar que los nombres tienen como mínimo 2 tokens y máximo 4 tokens. Además, se pudo observar que los nombres contienen caracteres no alfabéticos (` `, `-`).

In [9]:
min_words_name = get_min_token_count_by_column("Nombre")
max_words_name = get_max_token_count_by_column("Nombre")   

print(f"Mínimo: {min_words_name}, Máximo: {max_words_name}")


Mínimo: 2, Máximo: 4


In [10]:
names_chars = set("".join(dataset["Nombre"].astype(str)))
non_alpha_chars = {c for c in names_chars if not ALPHABET_RE.fullmatch(c)}

print("Caracteres no alfabéticos en nombres:", non_alpha_chars)

Caracteres no alfabéticos en nombres: {'-', ' '}


Por lo que el siguiente patrón de expresiones regulares fue diseñado para detectar nombres propios en español, incluso cuando ya han sido ofuscados e identificados como un token de **Nombre** con la longitud de tokens explicada anteriormente. También maneja correctamente nombres compuestos separados por guiones o espacios. Además, permite partículas en minúscula opcionales (como "de", "la", etc.) y otros nombres o placeholders que pueden aparecer en los nombres.

In [11]:
NAME_RE = re.compile(r"""
(?P<name>
    (?:
        <NOMBRE:\d+>                              
        |
        [A-ZÁÉÍÓÚÜÑ][a-záéíóúüñ']+                 # Nombre
    )
    (?:
        (?:\s+|-)                                  # Separador: espacio o guion
        (?:
            [a-záéíóúüñ']{1,3}\s+                  # partícula en minúscula opcional (como "de", "la", etc.)
        )?
        (?:
            <NOMBRE:\d+>                          
            |
            [A-ZÁÉÍÓÚÜÑ][a-záéíóúüñ]+             # Nombre o apellido
        )
    ){1,3}                                         # de 1 a 3 nombres extras
)
""", re.VERBOSE)


### Correo

Los correos electrónicos tienen un patrón bastante estándar, que incluye un nombre de usuario seguido por el símbolo `@` y un dominio.

In [12]:
EMAIL_RE   = re.compile(r"""
    (?P<email>
        [a-zA-Z0-9-]+                   # Nombre de usuario
        @
        [a-zA-Z0-9-]+                   # Dominio
        \.
        [a-zA-Z]{2,}            
    )
""", re.VERBOSE)

### Teléfono

Como se puede apreciar, en los telefonos hay minimo 1 token y máximo 5 tokens, de los cuales, su longitud mínima es de 2 caracteres y la máxima de 12 caracteres. Además, pueden tener un código de país opcional el cual puede iniciar por `+` o `(`

In [13]:
first_char_phone = set(dataset["Teléfono"].astype(str).str[0])
print("Primer caracter en los teléfonos:", first_char_phone)

Primer caracter en los teléfonos: {'9', '(', '+'}


In [14]:
min_words_phone = get_min_token_count_by_column("Teléfono")
max_words_phone = get_max_token_count_by_column("Teléfono")   

print(f"Mínimo: {min_words_phone}, Máximo: {max_words_phone}")

Mínimo: 1, Máximo: 5


In [15]:
tokens_len_phone = dataset["Teléfono"].astype(str).str.split().explode().str.len()

min_len_token_phone = tokens_len_phone.min()
max_len_token_phone = tokens_len_phone.max()

print(f"Longitud mínima de token en Teléfono: {min_len_token_phone}, Longitud máxima: {max_len_token_phone}")

Longitud mínima de token en Teléfono: 2, Longitud máxima: 12


In [16]:
PHONE_RE = re.compile(r"""
(?P<phone>
    (?:\+?\(?\d{1,3}\)?[\s-]*)?          # Código de país
    \d{2,12}                             # Primer bloque de dígitos
    (?:[\s-]?\d{2,12}){0,4}              # Bloques extras
)
""", re.VERBOSE)

### Dirección

La estructura de las direcciones es más compleja, ya que pueden contener múltiples palabras, números de portal y otros elementos. En este caso, se ha definido un patrón que permite capturar direcciones con una longitud mínima de 2 tokens y máxima de 12 tokens, con una longitud mínima de token de 3 caracteres y una máxima de 10 caracteres. Además, se permite la presencia de comas y números de portal opcionales.

Además, la estructura tiene una primera palabra que debe comenzar con una letra mayúscula, y las siguientes palabras pueden ser en minúscula o mayúscula. También se permite la presencia de números de portal y otros elementos como "Piso 3", "Apto 4", etc.

In [17]:
min_words_address = get_min_token_count_by_column("Dirección")
max_words_address = get_max_token_count_by_column("Dirección")   

print(f"Mínimo: {min_words_address}, Máximo: {max_words_address}")

Mínimo: 3, Máximo: 10


In [18]:
tokens_len_address = dataset["Teléfono"].astype(str).str.split().explode().str.len()

min_len_token_address = tokens_len_address.min()
max_len_token_address = tokens_len_address.max()

print(f"Longitud mínima de token en Direccion: {min_len_token_address}, Longitud máxima: {max_len_token_address}")

Longitud mínima de token en Direccion: 2, Longitud máxima: 12


In [19]:
ADDRESS_RE = re.compile(r"""
(?P<address>
    (?:[A-ZÁÉÍÓÚÜÑ]             
        [\wÁÉÍÓÚÜÑáéíóúüñ'’\-.]*  
     | [a-záéíóúüñ]{1,3}        
    )
    (?:                      
        \s+
        (?:[A-ZÁÉÍÓÚÜÑ][\wÁÉÍÓÚÜÑáéíóúüñ'’\-.]*|[a-záéíóúüñ]{1,3})
    ){1,11}

    \s*,?\s*                    
    \d{1,4}[A-Za-zºª°]*        

    (?:                          
        \s+[A-Za-zÁÉÍÓÚÜÑáéíóúüñ'’\-.]+\s+\d{1,4}
    )*

    \s*,\s*                     
    [A-ZÁÉÍÓÚÜÑ]                
    [^,\.]+                      

    (?:\s*,\s*\d{5})?          

    \.?                         
)
(?=\s|$)                       
""", re.VERBOSE)

### Identificador

Finalmente, el identificador es un número de 8 a 9 caracteres, que pueden empezar yo terminar con una letra, pero siempre debe contener números. Este patrón es bastante específico y se ha diseñado para capturar identificadores que cumplen con estas características.

In [20]:
len_id = dataset["Identificación"].astype(str).str.len()

min_len_id = len_id.min()
max_len_id = len_id.max()

print(f"Mínimo: {min_len_id}, Máximo: {max_len_id}")

Mínimo: 8, Máximo: 9


In [21]:
ID_RE = re.compile(r"""
    (?P<id>
        [A-Z]*
        \d{7,9}
        [A-Z]*
    )
""", re.VERBOSE)

In [22]:
PATTERNS = [
    ("NOMBRE", NAME_RE),
    ("CORREO", EMAIL_RE),
    ("TELEFONO", PHONE_RE),
    ("DIRECCION", ADDRESS_RE),
    ("IDENTIFICADOR", ID_RE)
]

In [23]:
def redact(text, placeholder_fmt="<{label}>"):
    rep_map = {} 
    
    for label, pattern in PATTERNS:
        for m in pattern.finditer(text):
            original = m.group()
            if original not in rep_map:
                rep_map[original] = placeholder_fmt.format(
                    label=label)

    for original in sorted(rep_map, key=len, reverse=True):
        text = text.replace(original, rep_map[original])
        
    return text

In [24]:
dataset["Texto_redactado"] = dataset["Texto"].apply(redact)

#guardar el dataset con las columnas originales y la redactada
direct_cols = ["Nombre", "Correo", "Teléfono", "Dirección", "Identificación", "Texto"]
df_redacted = dataset.drop(columns=direct_cols)
df_redacted.to_csv("pii_dataset_redacted.csv", index=False)

### Discusión
El enfoque que se tuvo desde el inicio fue cuantificar las características reales del conjunto de datos-número de tokens, longitudes mínimas y máximas y caracteres atípicos en cada feature. Esto permitió ajustar con precisión los cuantificadores de cada patrón y así definir las regex. Dichos regex, como se puede ver, son bastante precisos y permiten capturar la mayoría de los casos sin generar falsos positivos.

No obstante, como parte del análisis, se encontraron las limitaciones de depender de regex. La primera es las posibles ambigüedades (como por ejemplo “Ávila” puede ser apellido o ciudad). El segundo es la escalabilidad, ya que la llegada de nuevos formatos de datos obligaría a reescribir reglas manualmente. Por último, también podrían aparecer falsos negativos con abreviaturas inusuales (“C/”, “Avd.”) y falsos positivos cuando secuencias numéricas largas se parezcan a teléfonos. 

Es por ello, que una posible solución sería combinar estas regex como filtrado inicial con algún modelo que esté entrenado para estos casos de PII. Además, como vimos en clase, las NERs pueden ser una herramienta poderosa para identificar entidades nombradas y así complementar el filtrado de regex en los casos que expliqué anteriormente.

Ahora bien en el tema ético, la ofuscación correcta de PII no es solo una buena práctica técnica, sino que también es una obligación legal. Si no se hace de forma correcta puede llevar a sanciones económicas o incluso, un daño reputacional. En este caso, podría ser dañino para el hospital, además de poner en riesgo a los pacientes frente a fraudes o discriminación. 

En conclusión, el proyecto demuestra que un análisis exploratorio riguroso y expresiones regulares cuidadosamente afinadas pueden brindar una anonimización efectiva dentro de los límites de la práctica solicitada. Sin embargo, para entornos de producción donde la escala y la variabilidad de los datos crecen, resulta aconsejable evolucionar hacia herramientas híbridas que mezclen regex, aprendizaje automático y validaciones especializadas, logrando así un equilibrio sostenible entre precisión, mantenimiento y escalabilidad.